# Extraire les intensités de la lésion sur un T1 d'un patient 

In [3]:
import SimpleITK as sitk

mask_path   = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_MASK.nii.gz"
t1_path     = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T1.nii.gz"
t2_path     = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T2.nii.gz"
flair_path  = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_FLAIR.nii.gz"

# Lecture
mask_img_orig  = sitk.ReadImage(mask_path)
t1_img_orig    = sitk.ReadImage(t1_path)
t2_img_orig    = sitk.ReadImage(t2_path)
flair_img_orig = sitk.ReadImage(flair_path)

# 1) Masque binaire -> entier (labels)
#    (>0 pour être sûr même si le masque est en float)
mask_bin   = mask_img_orig > 0          # image binaire
mask_label = sitk.Cast(mask_bin, sitk.sitkUInt8)  # 0 / 1, type entier

# 2) Images d'intensité -> float64 (type supporté)
t1_img    = sitk.Cast(t1_img_orig,    sitk.sitkFloat64)
t2_img    = sitk.Cast(t2_img_orig,    sitk.sitkFloat64)
flair_img = sitk.Cast(flair_img_orig, sitk.sitkFloat64)

stats_filter = sitk.LabelStatisticsImageFilter()

def get_stats(image, label_img, label=1):
    stats_filter.Execute(image, label_img)
    if not stats_filter.HasLabel(label):
        raise ValueError(f"Aucun voxel avec le label {label} dans le masque.")
    mean_val = stats_filter.GetMean(label)
    min_val  = stats_filter.GetMinimum(label)
    max_val  = stats_filter.GetMaximum(label)
    return mean_val, min_val, max_val

# Lésion = label 1 dans le masque
t1_mean, t1_min, t1_max = get_stats(t1_img,    mask_label, label=1)
t2_mean, t2_min, t2_max = get_stats(t2_img,    mask_label, label=1)
fl_mean, fl_min, fl_max = get_stats(flair_img, mask_label, label=1)

print("T1    -> mean:", t1_mean, "min:", t1_min, "max:", t1_max)
print("T2    -> mean:", t2_mean, "min:", t2_min, "max:", t2_max)
print("FLAIR -> mean:", fl_mean, "min:", fl_min, "max:", fl_max)

T1    -> mean: 131.95627358034247 min: 29.0 max: 200.0
T2    -> mean: 348.2055238463771 min: 181.0 max: 628.0
FLAIR -> mean: 215.18878784947276 min: 130.0 max: 269.0


# Sur tout les patients

In [ ]:
import os
import csv
import SimpleITK as sitk

# Dossier racine contenant train/
BASE_DIR = "../../27919209/MSLesSegDataset/train"

# Fonction de stats pour une image + masque
stats_filter = sitk.LabelStatisticsImageFilter()

def get_stats(image, label_img, label=1):
    stats_filter.Execute(image, label_img)
    if not stats_filter.HasLabel(label):
        raise ValueError(f"Aucun voxel avec le label {label} dans le masque.")
    mean_val = stats_filter.GetMean(label)
    min_val  = stats_filter.GetMinimum(label)
    max_val  = stats_filter.GetMaximum(label)
    return mean_val, min_val, max_val


# Fichier CSV de sortie
output_csv = "lesion_stats.csv"
with open(output_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["patient", "session", "modality", "mean", "min", "max"])

    # --- Boucle sur les patients ---
    for patient in sorted(os.listdir(BASE_DIR)):
        patient_dir = os.path.join(BASE_DIR, patient)
        if not os.path.isdir(patient_dir):
            continue

        # --- Boucle sur les sessions (T1, T2, T3...) ---
        for session in sorted(os.listdir(patient_dir)):
            session_dir = os.path.join(patient_dir, session)
            if not os.path.isdir(session_dir):
                continue

            # Préfixe supposé : P1_T1, P1_T2, etc.
            prefix = f"{patient}_{session}"

            mask_path  = os.path.join(session_dir, f"{prefix}_MASK.nii.gz")
            t1_path    = os.path.join(session_dir, f"{prefix}_T1.nii.gz")
            t2_path    = os.path.join(session_dir, f"{prefix}_T2.nii.gz")
            flair_path = os.path.join(session_dir, f"{prefix}_FLAIR.nii.gz")

            # Vérification basique que les fichiers existent
            if not (os.path.exists(mask_path) and os.path.exists(t1_path) \
                    and os.path.exists(t2_path) and os.path.exists(flair_path)):
                print(f"[WARN] Fichiers manquants pour {patient} / {session}, on saute.")
                continue

            print(f"Traitement {patient} / {session} ...")

            # Lecture
            mask_img_orig  = sitk.ReadImage(mask_path)
            t1_img_orig    = sitk.ReadImage(t1_path)
            t2_img_orig    = sitk.ReadImage(t2_path)
            flair_img_orig = sitk.ReadImage(flair_path)

            # Masque binaire -> labels entiers
            mask_bin   = mask_img_orig > 0
            mask_label = sitk.Cast(mask_bin, sitk.sitkUInt8)

            # Intensités -> float64
            t1_img    = sitk.Cast(t1_img_orig,    sitk.sitkFloat64)
            t2_img    = sitk.Cast(t2_img_orig,    sitk.sitkFloat64)
            flair_img = sitk.Cast(flair_img_orig, sitk.sitkFloat64)

            # Lésion = label 1
            try:
                t1_mean, t1_min, t1_max = get_stats(t1_img,    mask_label, label=1)
                t2_mean, t2_min, t2_max = get_stats(t2_img,    mask_label, label=1)
                fl_mean, fl_min, fl_max = get_stats(flair_img, mask_label, label=1)
            except ValueError as e:
                print(f"[WARN] {patient} / {session} : {e}")
                continue

            # Écriture dans le CSV
            writer.writerow([patient, session, "T1",    t1_mean, t1_min, t1_max])
            writer.writerow([patient, session, "T2",    t2_mean, t2_min, t2_max])
            writer.writerow([patient, session, "FLAIR", fl_mean, fl_min, fl_max])

print(f"Terminé. Résultats sauvegardés dans {output_csv}")


Traitement P1 / T1 ...
Traitement P1 / T2 ...
Traitement P1 / T3 ...
Traitement P10 / T1 ...
Traitement P10 / T2 ...
Traitement P11 / T1 ...
Traitement P11 / T2 ...
Traitement P12 / T1 ...
Traitement P12 / T2 ...
Traitement P12 / T3 ...
Traitement P12 / T4 ...
Traitement P13 / T1 ...
Traitement P13 / T2 ...
Traitement P14 / T1 ...
Traitement P14 / T2 ...
Traitement P14 / T3 ...
Traitement P14 / T4 ...
Traitement P15 / T1 ...
Traitement P16 / T1 ...
Traitement P17 / T1 ...
Traitement P18 / T1 ...
Traitement P19 / T1 ...
Traitement P19 / T2 ...
Traitement P19 / T3 ...
Traitement P19 / T4 ...
Traitement P2 / T1 ...
Traitement P2 / T2 ...
Traitement P2 / T3 ...
Traitement P2 / T4 ...
Traitement P20 / T1 ...
Traitement P20 / T2 ...
Traitement P20 / T3 ...
Traitement P21 / T1 ...
Traitement P22 / T1 ...
Traitement P22 / T2 ...
Traitement P23 / T1 ...
Traitement P24 / T1 ...
Traitement P25 / T1 ...
Traitement P26 / T1 ...
Traitement P27 / T1 ...
Traitement P28 / T1 ...
Traitement P28 / T2 ...

# pour un patient, classifier Hyper/Hypo en T1, T2, FLAIR

In [5]:
mask_path   = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_MASK.nii.gz"
t1_path     = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T1.nii.gz"
t2_path     = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T2.nii.gz"
flair_path  = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_FLAIR.nii.gz"

# Lecture
mask_img_orig  = sitk.ReadImage(mask_path)
t1_img_orig    = sitk.ReadImage(t1_path)
t2_img_orig    = sitk.ReadImage(t2_path)
flair_img_orig = sitk.ReadImage(flair_path)

# Masque de lésion binaire -> labels entiers (0/1)
lesion_bin   = mask_img_orig > 0
lesion_label = sitk.Cast(lesion_bin, sitk.sitkUInt8)

# Masque "cerveau" (tout ce qui a une intensité > 0 en T1)
brain_bin = t1_img_orig > 0

# Masque de référence = cerveau hors lésion
ref_bin   = sitk.And(brain_bin, sitk.Not(lesion_bin))
ref_label = sitk.Cast(ref_bin, sitk.sitkUInt8)

# Images d'intensité -> float64
t1_img    = sitk.Cast(t1_img_orig,    sitk.sitkFloat64)
t2_img    = sitk.Cast(t2_img_orig,    sitk.sitkFloat64)
flair_img = sitk.Cast(flair_img_orig, sitk.sitkFloat64)

stats_filter = sitk.LabelStatisticsImageFilter()

def get_mean(image, label_img, label=1):
    stats_filter.Execute(image, label_img)
    if not stats_filter.HasLabel(label):
        raise ValueError("Pas de voxels pour ce label.")
    return stats_filter.GetMean(label)

def classify(lesion_mean, ref_mean, tol_ratio=0.05):
    """
    tol_ratio = 5% de marge pour éviter de tout classer hyper/hypo juste à cause du bruit.
    """
    tol = tol_ratio * ref_mean
    if lesion_mean > ref_mean + tol:
        return "hypersignal"
    elif lesion_mean < ref_mean - tol:
        return "hyposignal"
    else:
        return "isosignal"

# ---- T1 ----
t1_lesion_mean = get_mean(t1_img, lesion_label, label=1)
t1_ref_mean    = get_mean(t1_img, ref_label,   label=1)
t1_class       = classify(t1_lesion_mean, t1_ref_mean)

# ---- T2 ----
t2_lesion_mean = get_mean(t2_img, lesion_label, label=1)
t2_ref_mean    = get_mean(t2_img, ref_label,   label=1)
t2_class       = classify(t2_lesion_mean, t2_ref_mean)

# ---- FLAIR ----
fl_lesion_mean = get_mean(flair_img, lesion_label, label=1)
fl_ref_mean    = get_mean(flair_img, ref_label,   label=1)
fl_class       = classify(fl_lesion_mean, fl_ref_mean)

print("T1   : lesion_mean =", t1_lesion_mean, "ref_mean =", t1_ref_mean, "=>", t1_class)
print("T2   : lesion_mean =", t2_lesion_mean, "ref_mean =", t2_ref_mean, "=>", t2_class)
print("FLAIR: lesion_mean =", fl_lesion_mean, "ref_mean =", fl_ref_mean, "=>", fl_class)

T1   : lesion_mean = 131.95627358034247 ref_mean = 130.46934342308546 => isosignal
T2   : lesion_mean = 348.2055238463771 ref_mean = 252.0318421292086 => hypersignal
FLAIR: lesion_mean = 215.18878784947276 ref_mean = 118.31044812191666 => hypersignal
